# Load CSVs (one-to-many) to Azure AI Search

In this Jupyter Notebook, we create and run steps to index a CSV file in which each row is an individual and independent record/document. Each row then becomes searchable in Azure AI Search. 
The reference documentation can be found at [Indexing blobs and files to produce multiple search documents](https://learn.microsoft.com/en-us/azure/search/search-howto-index-one-to-many-blobs).

By default, an indexer will treat the contents of a blob or file as a single search document. If you want a more granular representation in a search index, you can set `parsingMode` values to create multiple search documents from one blob or file.

We are going to be using the same private Blob Storage account but a different container that has abstracts of 90k Medical publications about COVID-19 published in 2020-2022. This file is a subset of a much bigger dataset (770k articles) called CORD19 [HERE](https://github.com/allenai/cord19)

In [1]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv("credentials.env")

# Name of the container in your Blob Storage Datasource ( in credentials.env)
BLOB_CONTAINER_NAME = "ncc"

In [2]:
# Define the names for the data source, index and indexer
datasource_name = "srch-datasource-csv"
skillset_name = "srch-skillset-csv"
index_name = "srch-index-csv"
indexer_name = "srch-indexer-csv"

In [3]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## Create Data Source (Blob container with the CORD19 CSV data file)

In [ ]:
# Create a data source

datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities of one-to-many.",
    "type": "azureblob",
    "credentials": {
        "connectionString": os.environ['BLOB_CONNECTION_STRING']
    },
    "container": {
        "name": BLOB_CONTAINER_NAME
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)
r.text

## Inspect CSV file so we can understand the column types before creating the Index

In our private dataset we have place a smaller version of the original the metadata.csv file in the cord19 dataset. 
Let's see what the file looks like:

In [6]:
import pandas as pd
# Load the dataset with streaming enabled
import itertools


data = load_dataset("NbAiLab/NCC", streaming=True)

# Take only a small amount of data, e.g., 100 samples
data = pd.DataFrame(list(itertools.islice(data['train'], 1000)))

In [ ]:
print(data.head())

In [ ]:
# Define the functions for formatting
def make_clickable(address):
    '''Make the url clickable'''
    return f'<a href="{address}">{address}</a>'

def preview(text):
    '''Show only a preview of the text data.'''
    if isinstance(text, str):
        return text[:30] + '...'
    return text

# Define the format dictionary
format_ = {
    'id': preview,
    'doc_type': preview,
    'publish_year': preview,
    'lang_fasttext': preview,
    'lang_fasttext_conf': preview,
    'text': preview
}

# Apply the formatting and display the DataFrame
formatted_data = data.head().style.format(format_)
formatted_data

## Create the Index
In Azure AI Search, both blob indexers and file indexers support a delimitedText parsing mode for CSV files that treats each line in the CSV as a separate search document.

### **Important**:
As you can see below and from the prior Notebook, there are 6 mandatory fields in the schema: `id, title, name, location, chunk, chunkVector`. These fields must exist in any index that you create regardles of the datasource. Any additional fields are good to add so the engine can search relevant documents, however the mandatory fields must exist for all the code downstream work with no issues.

In [ ]:
index_payload = {
    "name": index_name,
    "fields": [
        {
            "name": "auto_key", 
            "type": "Edm.String", 
            "key": "true", 
            "analyzer": "keyword",  # Adding the keyword analyzer as required for the key field
            "searchable": "true",  # Key fields need to be searchable with the keyword analyzer
            "retrievable": "true", 
            "sortable": "false", 
            "filterable": "false", 
            "facetable": "false"
        },
        {
            "name": "id", 
            "type": "Edm.String", 
            "searchable": "true", 
            "retrievable": "true", 
            "sortable": "false", 
            "filterable": "true", 
            "facetable": "false"
        },
        {
            "name": "title", 
            "type": "Edm.String", 
            "analyzer": "standard",  # Set an analyzer for the title field (e.g., standard or custom if needed)
            "searchable": "true", 
            "retrievable": "true", 
            "sortable": "false", 
            "filterable": "false", 
            "facetable": "false"
        },
        {
            "name": "name", 
            "type": "Edm.String", 
            "searchable": "true", 
            "retrievable": "true", 
            "sortable": "false", 
            "filterable": "false", 
            "facetable": "false"
        },
        {
            "name": "location", 
            "type": "Edm.String", 
            "searchable": "true", 
            "retrievable": "true", 
            "sortable": "false", 
            "filterable": "false", 
            "facetable": "false"
        },
        {
            "name": "chunk", 
            "type": "Edm.String", 
            "searchable": "true", 
            "retrievable": "true", 
            "sortable": "false", 
            "filterable": "false", 
            "facetable": "false"
        },
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "dimensions": 3072,
            "vectorSearchProfile": "myprofile",
            "searchable": "true",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
            "facetable": "false"
        }
    ],
    "vectorSearch": {
        "algorithms": [
            {
                "name": "myalgo",
                "kind": "hnsw"
            }
        ],
        "vectorizers": [
            {
                "name": "openai",
                "kind": "azureOpenAI",
                "azureOpenAIParameters":
                {
                    "resourceUri" : os.environ['AZURE_OPENAI_ENDPOINT'],
                    "apiKey" : os.environ['AZURE_OPENAI_API_KEY'],
                    "deploymentId" : os.environ['EMBEDDING_DEPLOYMENT_NAME'],
                    "modelName" : os.environ['EMBEDDING_DEPLOYMENT_NAME'],
                }
            }
        ],
        "profiles": [
            {
                "name": "myprofile",
                "algorithm": "myalgo",
                "vectorizer":"openai"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ]
                }
            }
        ]
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)

print(json.dumps(index_payload))
print(r.status_code)
print(r.ok)
print(r.text)


## Create Skillset - Text Splitter, Language Detection
We don't need to use the OCR and Merge skill, since we know for sure that this is only text parsing. We will only use the SplitSkill and the AzureOpenAIEmbeddingSkill.

In [ ]:
skillset_payload = {
    "name": skillset_name,
    "description": "e2e Skillset for RAG - One to Many",
    "skills": [
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "context": "/document",
            "textSplitMode": "pages",
            "maximumPageLength": 5000,
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/text"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "chunks"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.AzureOpenAIEmbeddingSkill",
            "description": "Azure OpenAI Embedding Skill",
            "context": "/document/chunks/*",
            "resourceUri": os.environ['AZURE_OPENAI_ENDPOINT'],
            "apiKey": os.environ['AZURE_OPENAI_API_KEY'],
            "deploymentId": os.environ['EMBEDDING_DEPLOYMENT_NAME'],
            "modelName" : os.environ['EMBEDDING_DEPLOYMENT_NAME'],
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/chunks/*"
                }
            ],
            "outputs": [
                {
                    "name": "embedding",
                    "targetName": "vector"
                }
            ]
        }
    ],
    "indexProjections": {
        "selectors": [
            {
                "targetIndexName": index_name,
                "parentKeyFieldName": "id",  # id links the chunks to the parent document
                "sourceContext": "/document/chunks/*",
                "mappings": [
                    {
                        "name": "id",
                        "source": "/document/id"  # Ensure that the parent id is passed here
                    },
                    {
                        "name": "title",
                        "source": "/document/id"
                    },
                    {
                        "name": "name",
                        "source": "/document/doc_type"
                    },
                    {
                        "name": "location",
                        "source": "/document/publish_year"
                    },
                    {
                        "name": "chunk",
                        "source": "/document/chunks/*"
                    },
                    {
                        "name": "chunkVector",
                        "source": "/document/chunks/*/vector"
                    }
                ]
            }
        ],
        "parameters": {
            "projectionMode": "skipIndexingParentDocuments"
        }
    },
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": os.environ['COG_SERVICES_NAME'],
        "key": os.environ['COG_SERVICES_KEY']
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)

print(r.status_code)
print(r.ok)
print(r.text)


## Create and Run the Indexer - (runs the pipeline)
To create one-to-many indexers with CSV blobs, create or update an indexer definition with the delimitedText parsing mode

In [ ]:
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "schedule" : { "interval" : "PT30M"}, # How often do you want to check for new content in the data source
    "fieldMappings": [
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "name"
        },
        {
          "sourceFieldName" : "url",
          "targetFieldName" : "location"
        }
    ],
    "outputFieldMappings":
    [],
    "parameters" : { 
        "configuration" : { 
            "dataToExtract": "contentAndMetadata",
            "parsingMode" : "delimitedText", 
            "firstLineContainsHeaders" : True,
            "delimitedTextDelimiter": ","
        } 
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)
r.text

In [ ]:
# Optionally, get indexer status to confirm that it's running
try:
    r = requests.get(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name +
                     "/status", headers=headers, params=params)
    print(json.dumps(r.json(), indent=1))
    print(r.status_code)
    print("Status:",r.json().get('lastResult').get('status'))
    print("Items Processed:",r.json().get('lastResult').get('itemsProcessed'))
    print(r.ok)
    r.text
except Exception as e:
    print("Wait a few seconds until the process starts and run this cell again.")
    print(e)

**When the indexer finishes running we will have all 90,000 rows indexed properly as separate documents in our Search Engine!.**

# Reference

- https://learn.microsoft.com/en-us/azure/search/search-howto-index-csv-blobs



# NEXT
Now that we have two separate indexes loaded with two different types of information, In the next notebook 3, we will do a Multi-Index query, sort the results based on the reranker semantic score of Azure AI Search, and then use OpenAI to understand the results and give the best answer possible